## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

[747, 984, 103, 689, 174]

In [3]:
data = range(1,1001)
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data,10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [8]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [9]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [10]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([1.97062364e-01, 1.49054096e+00, 2.59615032e+00, 1.51959816e+00,
       2.85318621e+00, 4.97307632e+00, 1.05516026e+00, 5.69913821e+00,
       8.80720458e+00, 4.15283138e+00, 4.09295272e+00, 4.39355430e+00,
       7.36628637e+00, 1.09479443e+01, 8.84609677e+00, 1.03569018e+01,
       1.13513525e+01, 6.45207777e+00, 3.03471695e-01, 1.05602198e+01,
       6.79847078e+00, 1.42602775e+01, 6.90013313e+00, 1.52017803e+00,
       1.13281279e+01, 1.56873581e+01, 2.38396233e+00, 2.37972870e+01,
       1.50271305e+01, 6.74828283e-01, 8.29577429e+00, 7.24467071e-01,
       3.18693961e+01, 2.82777199e+01, 3.08848917e+01, 3.11506901e+01,
       3.54689296e+01, 3.21628203e+01, 3.53450030e+01, 2.53947817e+01,
       2.38819894e+01, 8.91554183e+00, 1.00656093e+01, 2.50382617e+01,
       3.61580029e+01, 1.63356728e+01, 4.32142005e+01, 3.29662087e+01,
       2.69731435e+01, 3.93667386e+01, 2.09984220e+01, 2.74671223e+01,
       4.64690388e+01, 7.28482520e+00, 2.12060235e+01, 4.13044164e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [12]:
price_items = sc.parallelize(sales_figures, 10)
price_items.collect()


[0.19706236404809108,
 1.4905409636415599,
 2.5961503153803935,
 1.519598159462698,
 2.8531862065838247,
 4.97307631691074,
 1.0551602573712595,
 5.69913820917325,
 8.80720458387291,
 4.1528313756966195,
 4.092952723311287,
 4.39355429786068,
 7.366286367238878,
 10.947944341620524,
 8.846096770530298,
 10.356901756767192,
 11.351352459160243,
 6.452077769629312,
 0.30347169453348277,
 10.560219812704549,
 6.798470782398283,
 14.260277501798972,
 6.900133130297514,
 1.5201780321100173,
 11.328127916912257,
 15.687358073005248,
 2.3839623250885262,
 23.797286956896265,
 15.027130488715285,
 0.6748282830111274,
 8.295774291853798,
 0.724467071156905,
 31.869396068319094,
 28.277719943533548,
 30.884891677931844,
 31.15069010550153,
 35.46892957854014,
 32.162820293934566,
 35.345002960973524,
 25.39478168417418,
 23.88198941372195,
 8.915541833755231,
 10.065609323223578,
 25.038261701761414,
 36.1580029314321,
 16.33567278015423,
 43.21420048456054,
 32.96620866312573,
 26.9731434831105

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [13]:
def sales_tax(num):
    return num*.92

revenue_minus_tax = price_items.map(sales_tax)


Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [14]:
revenue_minus_tax.take(5)

[0.1812973749242438,
 1.371297686550235,
 2.3884582901499622,
 1.398030306705682,
 2.6249313100571188]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [15]:
discounted = revenue_minus_tax.map(lambda x: x*.9)

In [16]:
discounted.take(10)

[0.16316763743181942,
 1.2341679178952116,
 2.149612461134966,
 1.2582272760351139,
 2.362438179051407,
 4.1177071904020925,
 0.873672693103403,
 4.718886437195451,
 7.292365395446771,
 3.438544379076801]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [17]:
price_items.map(sales_tax).map(lambda x : x*0.9).takeOrdered(15)

[0.07359008567385904,
 0.092548999764619,
 0.16316763743181942,
 0.25127456307372376,
 0.25787263905295277,
 0.2823077121740356,
 0.31210856298001455,
 0.3441955419692355,
 0.5587578183332136,
 0.5998587349179174,
 0.873672693103403,
 1.2287749663584937,
 1.2341679178952116,
 1.2582272760351139,
 1.2587074105870943]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [18]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[9] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [19]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.19706236404809108, 0.16316763743181942), (1.4905409636415599, 1.2341679178952116), (2.5961503153803935, 2.149612461134966), (1.519598159462698, 1.2582272760351139), (2.8531862065838247, 2.362438179051407), (4.97307631691074, 4.1177071904020925), (1.0551602573712595, 0.873672693103403), (5.69913820917325, 4.718886437195451), (8.80720458387291, 7.292365395446771), (4.1528313756966195, 3.438544379076801)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [20]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.19706236404809108, 0.16316763743181942, 1.4905409636415599, 1.2341679178952116, 2.5961503153803935, 2.149612461134966, 1.519598159462698, 1.2582272760351139, 2.8531862065838247, 2.362438179051407]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [22]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

265

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [23]:
selected_items.reduce(lambda x,y: x+y)

120147.71841621023

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [24]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(39, 305.80784086290726),
 (32, 322.69623457094235),
 (33, 320.31295903420903),
 (22, 341.2156243367896),
 (34, 315.74388598895206),
 (15, 355.54201913230065),
 (8, 302.64017387393625)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [25]:
# calculate how much each user spent
user_spend = sales_data.reduceByKey(lambda x,y: x+y)
user_spend.take(10)

[(50, 2301.8660551600137),
 (20, 1585.0289684492168),
 (40, 2759.9462155474794),
 (10, 1420.5001484465492),
 (30, 565.2317594546638),
 (1, 2606.5540114669056),
 (41, 1710.9679977562014),
 (21, 2293.840815377502),
 (31, 4381.652052226647),
 (11, 3151.5161558307627)]

In [26]:
# sort the users from highest to lowest spenders
user_spend.sortBy(lambda x: x[1],ascending = False).collect()


[(13, 4808.374945322821),
 (31, 4381.652052226647),
 (17, 4360.516988391206),
 (39, 3983.685703924518),
 (37, 3954.9786558701912),
 (48, 3943.961053452435),
 (12, 3738.8128610525164),
 (16, 3396.3175839004057),
 (45, 3261.527257429066),
 (43, 3249.3438339419226),
 (11, 3151.5161558307627),
 (8, 3123.6590122249654),
 (24, 3026.7431115676395),
 (33, 2994.0433456863493),
 (6, 2908.4471709080285),
 (49, 2764.42495370076),
 (40, 2759.9462155474794),
 (42, 2741.883585693259),
 (15, 2650.2849427783854),
 (1, 2606.5540114669056),
 (28, 2576.7166685243856),
 (26, 2419.3914105834447),
 (35, 2401.086157638577),
 (50, 2301.8660551600137),
 (21, 2293.840815377502),
 (36, 2207.259984244047),
 (18, 2180.235767258393),
 (38, 2178.2515273232584),
 (9, 2155.1619294143916),
 (47, 2118.497801813959),
 (27, 2103.8326069231234),
 (25, 1929.8859821632834),
 (2, 1892.0976797481944),
 (19, 1834.8859267359069),
 (32, 1793.7753720695441),
 (7, 1721.19234927032),
 (41, 1710.9679977562014),
 (5, 1661.7936768480172

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [27]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(5, 10),
 (38, 10),
 (43, 8),
 (47, 8),
 (49, 8),
 (8, 8),
 (4, 7),
 (35, 7),
 (28, 7),
 (34, 7),
 (40, 7),
 (18, 7),
 (29, 6),
 (26, 6),
 (2, 6),
 (13, 6),
 (1, 6),
 (22, 6),
 (25, 6),
 (24, 6),
 (37, 6),
 (39, 5),
 (12, 5),
 (6, 5),
 (9, 5),
 (20, 5),
 (30, 5),
 (19, 5),
 (48, 5),
 (10, 5),
 (45, 5),
 (21, 5),
 (15, 5),
 (7, 5),
 (42, 4),
 (17, 4),
 (16, 4),
 (41, 4),
 (50, 4),
 (3, 4),
 (11, 4),
 (14, 3),
 (27, 3),
 (23, 3),
 (31, 3),
 (33, 3),
 (46, 3),
 (36, 2),
 (32, 2),
 (44, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
